In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
import pandas as pd
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
from my_tools import plot_dic, file_dic
%matplotlib qt
import matplotlib.dates as mdates

In [2]:
#set path
path = '/media/onno/Volume/GFS_T850/'

In [4]:
coordinatez = [
(62,54,0,8),#Germany
(54,46,0,8), #Spain
(46,38,0,8), #Ukraine
(62,54,8,16),#Germany
(54,46,8,16), #Spain
(46,38,8,16), #Ukraine
(62,54,16,24),#Germany
(54,46,16,24), #Spain
(46,38,16,24)] #Ukraine
# #loading GFS and ERA5 data for specific box 

#Choose Forecast model: GFS or ERA5RF
fcst_modelz = ['GFS','ERA5RF']
for fcst_model in fcst_modelz:
    for lat_0,lat_1,lon_0,lon_1 in coordinatez:
        #open anomaly files
        file_anom_p60 = 'era51_fldmean_mars_t850_79-19_24hourly_lon_{}_{}_lat_{}_{}_anom_from_smoothed_p60.nc'.format(lon_0,lon_1-2,lat_0-2,lat_1)
        file_anom_p40 = 'era51_fldmean_mars_t850_79-19_24hourly_lon_{}_{}_lat_{}_{}_anom_from_smoothed_p40.nc'.format(lon_0,lon_1-2,lat_0-2,lat_1)
        anom_p60 = xr.open_dataset(path+file_anom_p60).t.squeeze()
        anom_p40 = xr.open_dataset(path+file_anom_p40).t.squeeze()
        #Process data for warm extremes
        #Calculate dates of persitent extremes that are 4 consecutive days or longer
        anom_avg = (anom_p60 < 0) & (anom_p40>0)
        iszero = np.concatenate(([0], np.equal(anom_avg.values, True).view(np.int8), [0]))
        abs_diff = np.abs(np.diff(iszero))
        # Runs start and end where absdiff is 1.
        idxz = np.where(abs_diff == 1)[0].reshape(-1, 2)
        idx_bool = (idxz[:,1]-idxz[:,0])>=4
        avg_idxz = idxz[idx_bool,:]
        avg_dates = np.empty(avg_idxz.shape,dtype=object)
        for i,avg_idx in enumerate(avg_idxz):
            avg_date = [str(anom_p60.time[avg_idx[0]].values),
                       str(anom_p60.time[avg_idx[1]].values-np.timedelta64(1,'D'))]
            avg_dates[i]=avg_date
        np.save(path+'dates/avg_p40_p60_lon_{}_{}_lat_{}_{}.npy'.format(lon_0,lon_1,lat_0,lat_1),avg_dates)
        np.savetxt(path+'dates/avg_p40_p60_lon_{}_{}_lat_{}_{}.txt'.format(lon_0,lon_1,lat_0,lat_1),avg_dates,fmt="%s")

        

FileNotFoundError: [Errno 2] No such file or directory: b'/media/onno/Volume/GFS_T850/era51_fldmean_mars_t850_79-19_24hourly_lon_0_6_lat_60_54_anom_from_smoothed_p60_ALL.nc'